In [104]:
import numpy as np
import csv
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.preprocessing import LabelEncoder


#reads in the data
data = pd.read_csv('movie_metadata.csv')

#removes the values that aren't numeric
to_drop = ['director_name', 'num_critic_for_reviews', 'actor_2_name','actor_1_name', 'movie_title', 'num_voted_users'
           , 'actor_3_name', 'plot_keywords', 'movie_imdb_link', 'num_user_for_reviews']

#makes the new data set without the to_drop colums
features_list = data.columns.difference(to_drop)
movie_data = data[features_list]
#print(np.sum(movie_num.isnull()))
#print(movie_data.content_rating.unique())
pd.options.mode.chained_assignment = None 

        
print('Done')

Done


# Changing Genres into individual Columns

In [105]:
genres = []

for string in movie_data['genres']:
    genre = string.split('|')
    genres = genres+genre
    
genres_set = set(genres)
print(genres_set)

{'Adventure', 'Music', 'Musical', 'History', 'Family', 'Biography', 'Romance', 'Sci-Fi', 'Game-Show', 'Comedy', 'Fantasy', 'Sport', 'Mystery', 'News', 'Film-Noir', 'Short', 'Western', 'Animation', 'Drama', 'Horror', 'Reality-TV', 'War', 'Documentary', 'Thriller', 'Action', 'Crime'}


In [106]:
genres_dict = dict()
for genre in genres_set:
    genres_dict[genre] = []
    
for string in movie_data.genres:
    genres = string.split('|')
    for genre in genres_set:
        if genre in genres:
            genres_dict[genre] = genres_dict[genre]+[1]
        else:
            genres_dict[genre] = genres_dict[genre]+[0]

#del movie_data['genres']
#print(genres_dict['Short'])

for genre in genres_set:
    series = pd.Series(genres_dict[genre])
    movie_data[genre] = series

#print(movie_data['Short'].values)    
print('Done')

# Deleting NaN's

In [107]:
NA_THRESH = 4

#deletes the row when there are more or equal to the threshod number of NaN's
def remove_too_many_NaN(data, threshold):
    print("Length before deletion: {}".format(len(data)))

    remove_indices = []
    for index, nNaN in data.isnull().sum(axis=1).iteritems():
        if nNaN >= threshold:
            remove_indices.append(index)
    
    # drop movies with too many NaNs
    data = data.drop(data.index[remove_indices])
    print("Length after deletion: {}".format(len(data)))
    
    return data

#returns length
movie_data = remove_too_many_NaN(movie_data, NA_THRESH)

Length before deletion: 5043
Length after deletion: 4900


# Changing Color to Numeric

In [108]:
#replaces the color value to 1
movie_data.color = movie_data.color.replace(to_replace = 'Color', value = 1)
movie_data.color = movie_data.color.replace(to_replace = 'NaN', value = 1)

#replaces the NaN or black and white values to 0
for item in new_color_column:
    if item != 1:
        movie_data.color = new_color_column.replace(to_replace = item, value = 0)
        
#movie_data.color = new_color_column 
#print(movie_data['color'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Changing Country to Numeric

In [109]:
#replaces the USA values to 1
movie_data.country = movie_data.country.replace(to_replace ='USA', value = 1)
movie_data.country = movie_data.country.replace(to_replace ='NaN', value = 1)
#replaces the NaN or non USA values to 0
for item in movie_data.country:
    if item != 1:
        movie_data.country = movie_data.country.replace(to_replace = item, value = 0)
       
#print(movie_data['country'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None

print('Done')

Done


# Changing Languange to Numeric

In [110]:
#replaces the English values to 1
new_language_column = movie_data.language.replace(to_replace = 'English', value = 1)
new_language_column = movie_data.language.replace(to_replace = 'NaN', value = 1)

#replaces the other values to 0
for item in new_language_column:
    if item != 1:
        new_language_column = new_language_column.replace(to_replace = item, value = 0)
        
movie_data.language = new_language_column 
#print(movie_data['language'])

#makes sure that there is nog error where it shouldn't be
pd.options.mode.chained_assignment= None
print('Done')

Done


# Chaning Content_Rating to Numeric

In [114]:
def content_to_numerical(data):
    data = data.replace(to_replace='G', value=0)
    data = data.replace(to_replace='PG', value=12)
    data = data.replace(to_replace='PG-13', value=13)
    data = data.replace(to_replace='R', value=17)
    data = data.replace(to_replace='NC-17', value=17)
    
    data = data.replace(to_replace='TV-PG', value=12)
    data = data.replace(to_replace='TV-MA', value=17)
    data = data.replace(to_replace='TV-G', value=0)
    data = data.replace(to_replace='TV-Y', value=0)
    data = data.replace(to_replace='TV-Y7', value=7)
    data = data.replace(to_replace='TV-14', value=14)
    
    data = data.replace(to_replace='Not Rated', value=0)
    data = data.replace(to_replace='Unrated', value=0)
    data = data.replace(to_replace='Approved', value=0)
    data = data.replace(to_replace='Passed', value=0)
    
    data = data.replace(to_replace='X', value=17)
    data = data.replace(to_replace='M', value=17)
    data = data.replace(to_replace='GP', value=12)
    
    return data

movie_data = content_to_numerical(movie_data)
print(movie_data['language'])

0       0
1       0
2       0
3       0
5       0
6       0
7       0
8       0
9       0
10      0
11      0
12      0
13      0
14      0
15      0
16      0
17      0
18      0
19      0
20      0
21      0
22      0
23      0
24      0
25      0
26      0
27      0
28      0
29      0
30      0
       ..
5008    0
5009    0
5010    0
5011    0
5012    0
5013    0
5014    0
5015    0
5016    0
5017    0
5018    0
5019    0
5021    0
5022    0
5023    0
5024    0
5025    0
5026    0
5027    0
5028    0
5029    0
5031    0
5033    0
5034    0
5035    0
5036    0
5037    0
5040    0
5041    0
5042    0
Name: language, dtype: int64


In [112]:
print(movie_data.color.describe())
print(movie_data.country.describe())
print(movie_data.language.describe())

count    4900.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: color, dtype: float64
count    4900.000000
mean        0.757347
std         0.428731
min         0.000000
25%         1.000000
50%         1.000000
75%         1.000000
max         1.000000
Name: country, dtype: float64
count    4900.0
mean        0.0
std         0.0
min         0.0
25%         0.0
50%         0.0
75%         0.0
max         0.0
Name: language, dtype: float64
